# Pré-Processamento UEL - Gerando dados para treinamento

In [ ]:
import pandas as pd
from itertools import cycle
import random

# 1. Carregar os arquivos
teste_ataque = pd.read_csv('data/cic_puro/teste_ataque_ordenado.csv', sep=';')
teste_normal = pd.read_csv('data/cic_puro/teste_sem_ataque_ordenado.csv', sep=';')
treino_ataque = pd.read_csv('data/cic_puro/treino_ataque_ordenado.csv', sep=';')
treino_normal = pd.read_csv('data/cic_puro/treino_sem_ataque_ordenado.csv', sep=';')


# 2. Concatenar para treino e teste
teste_full = pd.concat([teste_normal, teste_ataque], ignore_index=True)
treino_full = pd.concat([treino_normal, treino_ataque], ignore_index=True)

# 3. Separar normais e ataques
def prepare_data(df, max_per_attack=1000, max_normal=5000):
    normal = df[df['label'] == 0].sample(frac=1).reset_index(drop=True)  # embaralhar normais
    attacks = df[df['label'] == 1].reset_index(drop=True)

    # Agora limitar por tipo de ataque
    attack_types = {}
    for name, group in attacks.groupby('attack_name'):
        attack_types[name] = group.sample(n=min(len(group), max_per_attack)).reset_index(drop=True)

    # Limitar normais
    if max_normal is not None:
        normal = normal.sample(n=min(len(normal), max_normal)).reset_index(drop=True)

    return normal, attack_types

train_normal, train_attacks = prepare_data(treino_full, max_per_attack=1000, max_normal=10000)
test_normal, test_attacks = prepare_data(teste_full, max_per_attack=500, max_normal=5000)

# 4. Função para criar sequências aleatórias
def create_random_sequences(normal_df, attack_dict, min_seq=30, max_seq=150):
    final_rows = []
    
    normal_iter = normal_df.iterrows()
    attack_iters = {k: v.iterrows() for k, v in attack_dict.items()}
    attack_cycle = cycle(list(attack_iters.keys()))
    
    normal_remaining = True
    attack_remaining = True

    while normal_remaining or attack_remaining:
        choice = random.choice(['normal', 'attack'])  # Aleatoriamente decidir normal ou ataque primeiro
        
        if choice == 'normal' and normal_remaining:
            seq_len = random.randint(min_seq, max_seq)
            for _ in range(seq_len):
                try:
                    idx, row = next(normal_iter)
                    final_rows.append(row)
                except StopIteration:
                    normal_remaining = False
                    break
        
        elif choice == 'attack' and attack_remaining:
            attack_type = next(attack_cycle)
            seq_len = random.randint(min_seq, max_seq)
            for _ in range(seq_len):
                try:
                    idx, row = next(attack_iters[attack_type])
                    final_rows.append(row)
                except StopIteration:
                    # Se esgotar ataques desse tipo, remover do ciclo
                    del attack_iters[attack_type]
                    if attack_iters:
                        attack_cycle = cycle(list(attack_iters.keys()))
                    else:
                        attack_remaining = False
                    break
        else:
            # Se o tipo escolhido acabou, tenta o outro
            continue

    return pd.DataFrame(final_rows)

# 5. Criar datasets
train_final = create_random_sequences(train_normal, train_attacks, min_seq=30, max_seq=120)
test_final = create_random_sequences(test_normal, test_attacks, min_seq=30, max_seq=120)

# 6. Salvar
train_final.to_csv('treino_final_estratificado_random.csv', sep=';', index=False)
test_final.to_csv('teste_final_estratificado_random.csv', sep=';', index=False)

print('Arquivos treino_final_estratificado_random.csv e teste_final_estratificado_random.csv gerados com sequências aleatórias!')

Arquivos treino_final_estratificado_random.csv e teste_final_estratificado_random.csv gerados com sequências aleatórias!


In [ ]:
# Contar a quantidade de cada valor na coluna 'attack_name'
attack_counts_train = train_final['attack_name'].value_counts()
attack_counts_test = test_final['attack_name'].value_counts()

# Exibir os resultados
print('Tamanho:', len(train_final), 'Treino:', attack_counts_train)
print('Total de linhas no conjunto de treino:', len(train_final))

print('Tamanho:', len(test_final), 'Teste:', attack_counts_test)
print('Total de linhas no conjunto de teste:', len(test_final))

Tamanho: 13 Treino: attack_name
normal           8074
DrDoS_DNS        1000
DrDoS_NTP        1000
DrDoS_SNMP       1000
DrDoS_UDP        1000
TFTP             1000
UDP-lag           885
DrDoS_SSDP        822
DrDoS_NetBIOS     726
DrDoS_MSSQL       687
DrDoS_LDAP        592
Syn               237
WebDDoS           125
Name: count, dtype: int64
Tamanho: 8 Teste: attack_name
normal     5000
LDAP        500
MSSQL       500
NetBIOS     500
Syn         500
UDP         500
UDPLag      470
Portmap     449
Name: count, dtype: int64


# Modelos

## LSTM

In [1]:
from models.LSTM import LSTM
from models.Sequence import SequenceDataset
from torch.utils.data import DataLoader
import torch

# Configurações gerais
SEED = 42
torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Parâmetros do dataset e modelo
input_size      = 9
hidden_size     = 256
num_layers      = 3
output_size     = 2
batch_size      = 128
sequence_length = 50
column_to_remove= 'attack_name'

Usando dispositivo: cuda


### Treinamento

In [ ]:
# Criar os datasets
train_dataset = SequenceDataset(
    path             = 'data/dataset/treino_final_estratificado_random.csv',
    sequence_length  = sequence_length,
    column_to_remove = column_to_remove,
    normalize        = True,
    mode             = 'lstm'
)
test_dataset = SequenceDataset(
    path             = 'data/dataset/teste_final_estratificado_random.csv',
    sequence_length  = sequence_length,
    column_to_remove = column_to_remove,
    normalize        = True,
    mode             = 'lstm'
)

print(f"Total de amostras no treino: {len(train_dataset)}")
print(f"Total de amostras no teste:  {len(test_dataset)}")
print("Train Shape:", train_dataset.sequences.shape)
print("Test  Shape:", test_dataset.sequences.shape)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size)
print(f"Batches treino: {len(train_loader)}, teste: {len(test_loader)}")

# Instanciar o modelo
model = LSTM(input_size=input_size,
             hidden_size=hidden_size,
             num_layers=num_layers,
             output_size=output_size).to(device)
print(model)

Usando dispositivo: cuda
Total de amostras no treino: 17099
Total de amostras no teste:  8370
Train Shape: torch.Size([17099, 50, 9])
Test  Shape: torch.Size([8370, 50, 9])
Batches treino: 134, teste: 66
LSTM(
  (lstm1): LSTM(9, 128, num_layers=3, batch_first=True)
  (lstm2): LSTM(128, 256, num_layers=3, batch_first=True, dropout=0.2)
  (lstm3): LSTM(256, 128, num_layers=3, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=128, out_features=2, bias=True)
)


In [2]:
# Treina e salva
model.train_model(
    train_loader,
    test_loader,
    device=device,
    epochs=50,
    lr=1e-3,
    save_dir='output/LSTM',
    threshold=0.87
)

Ep 1/50 — Train Loss: 0.3468 — Val Acc: 0.9004
→ Checkpoint salvo: LSTM_0.9004.pth
Ep 2/50 — Train Loss: 0.1354 — Val Acc: 0.9041
→ Checkpoint salvo: LSTM_0.9041.pth
Ep 3/50 — Train Loss: 0.0992 — Val Acc: 0.9016
Ep 4/50 — Train Loss: 0.0851 — Val Acc: 0.9097
→ Checkpoint salvo: LSTM_0.9097.pth
Ep 5/50 — Train Loss: 0.0700 — Val Acc: 0.9075
Ep 6/50 — Train Loss: 0.0839 — Val Acc: 0.8986
Ep 7/50 — Train Loss: 0.1061 — Val Acc: 0.9097
Ep 8/50 — Train Loss: 0.0886 — Val Acc: 0.9220
→ Checkpoint salvo: LSTM_0.9220.pth
Ep 9/50 — Train Loss: 0.0819 — Val Acc: 0.9139
Ep 10/50 — Train Loss: 0.0739 — Val Acc: 0.9211
Ep 11/50 — Train Loss: 0.0702 — Val Acc: 0.8742
Ep 12/50 — Train Loss: 0.0818 — Val Acc: 0.9234
→ Checkpoint salvo: LSTM_0.9234.pth
Ep 13/50 — Train Loss: 0.0692 — Val Acc: 0.9223
Ep 14/50 — Train Loss: 0.0808 — Val Acc: 0.9211
Ep 15/50 — Train Loss: 0.0655 — Val Acc: 0.9234
Ep 16/50 — Train Loss: 0.0633 — Val Acc: 0.9182
Ep 17/50 — Train Loss: 0.0534 — Val Acc: 0.9287
→ Checkpoint 

LSTM(
  (lstm1): LSTM(9, 128, num_layers=3, batch_first=True)
  (lstm2): LSTM(128, 256, num_layers=3, batch_first=True, dropout=0.2)
  (lstm3): LSTM(256, 128, num_layers=3, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=128, out_features=2, bias=True)
)

### Avaliação

In [ ]:
test_dataset = SequenceDataset(
    path             = 'data/dataset/teste_final_estratificado_random.csv',
    sequence_length  = sequence_length,
    column_to_remove = column_to_remove,
    normalize        = True,
    mode             = 'lstm'
)

test_loader  = DataLoader(test_dataset,  batch_size=batch_size)

# Instanciar o modelo e mover para device
model = LSTM(input_size, hidden_size, num_layers, output_size).to(device)

# Carregar o checkpoint salvo 
checkpoint_path = 'output/LSTM/LSTM_0.9470.pth'
state = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(state)
print(f"Checkpoint '{checkpoint_path}' carregado com sucesso!")

# Avaliar o modelo
model.evaluate(test_loader, device=device)


Checkpoint 'output/LSTM/LSTM_0.9470.pth' carregado com sucesso!

=== Classification Report ===
              precision    recall  f1-score   support

           0       0.96      0.95      0.96      4965
           1       0.93      0.94      0.94      3405

    accuracy                           0.95      8370
   macro avg       0.94      0.95      0.95      8370
weighted avg       0.95      0.95      0.95      8370

Accuracy: 0.9469534050179211


## CNN

In [2]:
from models.CNN import CNN
from models.Sequence import SequenceDataset
from torch.utils.data import DataLoader
import torch

SEED = 42
torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# hiperparâmetros
sequence_length = 70
column_to_remove = 'attack_name'
batch_size      = 64
input_channels  = None  
input_length    = sequence_length
num_classes     = 2
epochs          = 20
lr              = 1e-3
threshold       = 0.87
save_dir        = 'output/CNN'

Usando dispositivo: cuda


### Treinamento

In [3]:
train_dataset = SequenceDataset('data/dataset/treino_final_estratificado_random.csv', sequence_length, column_to_remove, normalize=True, mode='cnn1d')
valid_dataset = SequenceDataset('data/dataset/teste_final_estratificado_random.csv', sequence_length, column_to_remove, normalize=True, mode='cnn1d')

print(f"Total de amostras no conjunto de treino: {len(train_dataset)}")
print(f"Total de amostras no conjunto de teste: {len(valid_dataset)}")
print("Train Dataset Shape:", train_dataset.sequences.shape)
print("Test Dataset Shape:", valid_dataset.sequences.shape)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)

print(f"Total de batches no conjunto de treino: {len(train_loader)}")
print(f"Total de batches no conjunto de teste: {len(valid_loader)}")

# Modelo
input_channels = train_dataset.sequences.shape[1]
model = CNN(
    input_channels=input_channels,
    input_length=sequence_length,
    num_classes=num_classes
).to(device)
model.to(device)

Total de amostras no conjunto de treino: 17079
Total de amostras no conjunto de teste: 8350
Train Dataset Shape: torch.Size([17079, 9, 70])
Test Dataset Shape: torch.Size([8350, 9, 70])
Total de batches no conjunto de treino: 267
Total de batches no conjunto de teste: 131


CNN(
  (conv1): Conv1d(9, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
  (bn4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.4, inplace=False)
  (fc1): Linear(in_features=17920, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [4]:
# Treinar
model.train_model(
    train_loader,
    valid_loader,
    device=device,
    epochs=epochs,
    lr=lr,
    save_dir=save_dir,
    threshold=threshold
)

Ep 1/20 – Train Loss: 0.1183 – Val Acc: 0.9020
→ Checkpoint salvo: CNN_0.9020.pth
Ep 2/20 – Train Loss: 0.0532 – Val Acc: 0.9150
→ Checkpoint salvo: CNN_0.9150.pth
Ep 3/20 – Train Loss: 0.0374 – Val Acc: 0.9071
Ep 4/20 – Train Loss: 0.0334 – Val Acc: 0.9205
→ Checkpoint salvo: CNN_0.9205.pth
Ep 5/20 – Train Loss: 0.0236 – Val Acc: 0.8795
Ep 6/20 – Train Loss: 0.0205 – Val Acc: 0.8381
Ep 7/20 – Train Loss: 0.0182 – Val Acc: 0.9139
Ep 8/20 – Train Loss: 0.0141 – Val Acc: 0.9151
Ep 9/20 – Train Loss: 0.0120 – Val Acc: 0.9333
→ Checkpoint salvo: CNN_0.9333.pth
Ep 10/20 – Train Loss: 0.0077 – Val Acc: 0.9195
Ep 11/20 – Train Loss: 0.0159 – Val Acc: 0.9256
Ep 12/20 – Train Loss: 0.0068 – Val Acc: 0.8966
Ep 13/20 – Train Loss: 0.0082 – Val Acc: 0.8862
Ep 14/20 – Train Loss: 0.0057 – Val Acc: 0.9013
Ep 15/20 – Train Loss: 0.0037 – Val Acc: 0.9242
Ep 16/20 – Train Loss: 0.0074 – Val Acc: 0.9279
Ep 17/20 – Train Loss: 0.0062 – Val Acc: 0.9170
Ep 18/20 – Train Loss: 0.0018 – Val Acc: 0.9055
Ep 19

CNN(
  (conv1): Conv1d(9, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
  (bn4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.4, inplace=False)
  (fc1): Linear(in_features=17920, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

### Avaliação

In [4]:
test_dataset = SequenceDataset(
    path             = 'data/dataset/teste_final_estratificado_random.csv',
    sequence_length  = sequence_length,
    column_to_remove = column_to_remove,
    normalize        = True,
    mode             = 'cnn1d'
)

test_loader  = DataLoader(test_dataset,  batch_size=batch_size)

# Modelo
input_channels = test_dataset.sequences.shape[1]
model = CNN(
    input_channels=input_channels,
    input_length=sequence_length,
    num_classes=num_classes
).to(device)

# Carregar o checkpoint salvo 
checkpoint_path = 'output/CNN/CNN_0.9333.pth'
state = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(state)
print(f"Checkpoint '{checkpoint_path}' carregado com sucesso!")

# Avaliar o modelo
model.evaluate(test_loader, device=device)

Checkpoint 'output/CNN/CNN_0.9333.pth' carregado com sucesso!

=== Classification Report ===
              precision    recall  f1-score   support

           0       0.94      0.95      0.94      4965
           1       0.93      0.90      0.92      3385

    accuracy                           0.93      8350
   macro avg       0.93      0.93      0.93      8350
weighted avg       0.93      0.93      0.93      8350

Accuracy: 0.9332934131736527


## Hybrid V2

In [1]:
import torch
from torch.utils.data import DataLoader
from models.Sequence import SequenceDataset
from models.Hybrid import ModelHybridAttnSVM

# Fix seed e escolher device
SEED = 42
torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Hiperparâmetros
sequence_length = 70
column_to_remove = 'attack_name'
batch_size      = 64
lstm_hidden     = 64
lstm_layers     = 2
num_classes     = 2
pca_components  = 30
svm_C           = 1.0
epochs          = 50
lr              = 1e-3
threshold       = 0.87
save_dir        = 'output/Hybrid'

Usando dispositivo: cuda


### Treinamento

In [5]:
# Criar os datasets
train_dataset = SequenceDataset(
    path             = 'data/dataset/treino_final_estratificado_random.csv',
    sequence_length  = sequence_length,
    column_to_remove = column_to_remove,
    normalize        = True,
    mode             = 'lstm'
)
valid_loader = SequenceDataset(
    path             = 'data/dataset/teste_final_estratificado_random.csv',
    sequence_length  = sequence_length,
    column_to_remove = column_to_remove,
    normalize        = True,
    mode             = 'lstm'
)

print(f"Total de amostras no conjunto de treino: {len(train_dataset)}")
print(f"Total de amostras no conjunto de teste:  {len(valid_loader)}")
print("Train Dataset Shape:", train_dataset.sequences.shape)
print("Test  Dataset Shape:", valid_loader.sequences.shape)

# Criar os DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader  = DataLoader(valid_loader,  batch_size=batch_size)

print(f"Total de batches no treino: {len(train_loader)}")
print(f"Total de batches no teste:  {len(test_loader)}")

# Instanciar o modelo
n_features = train_dataset.sequences.shape[2]
model = ModelHybridAttnSVM(
    seq_len         = sequence_length,
    n_features      = n_features,
    lstm_hidden     = lstm_hidden,
    lstm_layers     = lstm_layers,
    num_classes     = num_classes,
    pca_components  = pca_components,
    svm_C           = svm_C
).to(device)
print(model)

Total de amostras no conjunto de treino: 17079
Total de amostras no conjunto de teste:  8350
Train Dataset Shape: torch.Size([17079, 70, 9])
Test  Dataset Shape: torch.Size([8350, 70, 9])
Total de batches no treino: 267
Total de batches no teste:  131
ModelHybridAttnSVM(
  (conv1): Conv1d(9, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv3): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (relu): ReLU()
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop): Dropout(p=0.5, inplace=False)
  (lstm): LSTM(128, 64, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=64, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)


In [6]:
# Treinamento
model.train_model(
    train_loader  = train_loader,
    valid_loader  = valid_loader,
    device        = device,
    epochs        = epochs,
    lr            = lr,
    save_dir      = save_dir,
    threshold     = threshold
)

Ep 1/50 – Train Loss: 0.2138 – Val Acc: 0.8995
→ Checkpoint salvo: Hybrid_0.90.pth
Ep 2/50 – Train Loss: 0.1052 – Val Acc: 0.9468
→ Checkpoint salvo: Hybrid_0.95.pth
Ep 3/50 – Train Loss: 0.0923 – Val Acc: 0.9504
→ Checkpoint salvo: Hybrid_0.95.pth
Ep 4/50 – Train Loss: 0.0823 – Val Acc: 0.9429
Ep 5/50 – Train Loss: 0.0738 – Val Acc: 0.9522
→ Checkpoint salvo: Hybrid_0.95.pth
Ep 6/50 – Train Loss: 0.0706 – Val Acc: 0.9559
→ Checkpoint salvo: Hybrid_0.96.pth
Ep 7/50 – Train Loss: 0.0663 – Val Acc: 0.9593
→ Checkpoint salvo: Hybrid_0.96.pth
Ep 8/50 – Train Loss: 0.0557 – Val Acc: 0.9577
Ep 9/50 – Train Loss: 0.0528 – Val Acc: 0.9537
Ep 10/50 – Train Loss: 0.0497 – Val Acc: 0.9505
Ep 11/50 – Train Loss: 0.0476 – Val Acc: 0.9532
Ep 12/50 – Train Loss: 0.0447 – Val Acc: 0.9505
Ep 13/50 – Train Loss: 0.0402 – Val Acc: 0.9505
Ep 14/50 – Train Loss: 0.0375 – Val Acc: 0.9538
Ep 15/50 – Train Loss: 0.0375 – Val Acc: 0.9513
Ep 16/50 – Train Loss: 0.0344 – Val Acc: 0.9416
Ep 17/50 – Train Loss: 0.

ModelHybridAttnSVM(
  (conv1): Conv1d(9, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv3): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (relu): ReLU()
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop): Dropout(p=0.5, inplace=False)
  (lstm): LSTM(128, 64, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=64, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [7]:
# 2) Treinar PCA + SVM sobre as features extraídas
model.train_svm(
    train_loader = valid_loader,
    device       = device,
    pca_path     = f"{save_dir}/pca.joblib",
    svm_path     = f"{save_dir}/hybrid_svm.joblib"
)

SVC(probability=True)

### Avaliação

In [3]:
# Dataset e DataLoader para teste
test_ds = SequenceDataset(
    path             = 'data/dataset/teste_final_estratificado_random.csv',
    sequence_length  = sequence_length,
    column_to_remove = column_to_remove,
    normalize        = True,
    mode             = 'lstm'
)
print(f"Total de amostras de teste: {len(test_ds)}")
print("Test shape:", test_ds.sequences.shape)

test_loader = DataLoader(
    test_ds,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

# Instanciar a classe Hybrid com os mesmos hiperparâmetros de treino
n_features = test_ds.sequences.shape[2]  # dimensão de features por timestep
model = ModelHybridAttnSVM(
    seq_len        = sequence_length,
    n_features     = n_features,
    lstm_hidden    = lstm_hidden,
    lstm_layers    = lstm_layers,
    num_classes    = num_classes,
    pca_components = 30,
    svm_C          = 1.0
).to(device)

# Carregar checkpoint CNN-LSTM salvo 
checkpoint_path = 'output/Hybrid/Hybrid_0.90.pth'
state = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(state)
print(f"Checkpoint '{checkpoint_path}' carregado com sucesso!")

# Avaliar pipeline completo (CNN→LSTM→PCA→SVM)
model.evaluate(
    loader    = test_loader,
    device    = device,
    pca_path  = 'output/Hybrid/pca.joblib',
    svm_path  = 'output/Hybrid/hybrid_svm.joblib'
)


Total de amostras de teste: 8350
Test shape: torch.Size([8350, 70, 9])
Checkpoint 'output/Hybrid/Hybrid_0.90.pth' carregado com sucesso!
              precision    recall  f1-score   support

           0       0.96      0.88      0.92      4965
           1       0.84      0.95      0.90      3385

    accuracy                           0.91      8350
   macro avg       0.90      0.92      0.91      8350
weighted avg       0.92      0.91      0.91      8350

Accuracy: 0.9094610778443114
